# Práctica 3 - Minería de Datos
## Datos Turismo
### Carla Piñol Galera, Aurora Larrosa Navarro, Ana Isabel Navarro Cortijo

En este notebook se realiza el análisis exploratorio de los datos proporcionados para llevar a cabo este trabajo. Se estudia la estructura de cada uno de ellos, las variables explicativas que los conforman y la información que se puede obtener a partir de ellos.


## Importar Librerías
En primer lugar se importan las librerías necesarias y se copia el directorio de trabajo de github en el que se encuentran los conjuntos de datos y las funciones auxiliares.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
import plotly.graph_objects as go

!git --version
!git clone https://github.com/carlapinol/Mineria-de-datos.git
%cd Mineria-de-datos

from carga import cargar_datos
from limpia import limpiar_datos
from plotea import plotear_datos
from spain import spain_heat_map

## Cargar los Datos
Utilizamos la función `cargar_datos` creada, para cargar los datasets proporcionados y almacenarlos en dataframes.

Hemos tomado otros datasets extra, para completar los datos proporcionados. Porcedemos a analizar cada dataset por separado.

In [ ]:
# Cargar los datasets
dataframes = cargar_datos('data')

### Distribución porcentual de los viajeros residentes por provincias y comunidades y ciudades autónomas de procedencia (2022)

**Columnas**
- Comunidades y ciudades autónomas de destino: CA de procedencia
- Comunidades y ciudades autónomas de procedencia: CA de destino
- Total: Porcentaje de población por provincia de destino

In [ ]:
url = "https://www.ine.es/jaxi/files/_px/csv_bdsc/tpx/Servicios_3061/Hosteleria_5881/EOH_2022_9924/01nad04.csv"
df1 = pd.read_csv(url, sep=";")
df1

Filtramos el dataset y nos quedamos con las columnas que nos interesan, ya que ['Destino'] y ['Procedencia'] no aportan información.

In [ ]:
df_new1 = limpiar_datos(df1, destiny='Valencia').iloc[:, -2:]

print("Total: % por CA de procedencia\n")
df_new1

Hacemos una representación para ver la distribución procentual de viajeros que han venido a la Comunidad Valenciana durante el año 2022 según su comunidad de procedencia.



In [ ]:
lista_total = df_new1['Total'].str.replace(',', '.').astype(float).tolist()[1:]
lista_total[-2]=lista_total[-2]+lista_total[-1]
lista_total.pop()

path = 'spain'
title = "Mapa porcentaje de viajeros según CA de procedencia"
spain_heat_map(lista_total, path, "bluyl",title, plot_type="plotly")

En el año 2022, la distribución porcentual de los viajeros que visitaron la Comunidad Valenciana según su comunidad de procedencia revela varias tendencias interesantes. La mayor proporción de visitantes provino de la propia Comunidad Valenciana, representando el 32.47% del total. Este dato sugiere una fuerte tendencia de turismo interno, donde los residentes de la Comunidad Valenciana eligen viajar dentro de su propia región.

La Comunidad de Madrid se destaca como la segunda mayor fuente de turistas con un 24.32%, lo que refleja la importancia de la capital y su área metropolitana como origen de viajeros hacia la Comunidad Valenciana. Cataluña también figura significativamente con un 9.79%, seguido de Andalucía con un 6.85%, lo que indica una considerable afluencia de turistas desde estas regiones.

Otras comunidades como Castilla - La Mancha (6.07%) y Murcia (3.74%) también contribuyen de manera notable al flujo turístico hacia la Comunidad Valenciana. Sin embargo, hay comunidades con una participación menor, como Asturias (0.99%), Baleares (1.15%), Canarias (0.61%), y La Rioja (0.58%). Las comunidades con la menor proporción de turistas son Ceuta (0.11%) y Melilla (0.03%).

Este análisis muestra que la Comunidad Valenciana no solo atrae a turistas de otras partes de España, sino que también mantiene un robusto turismo local. La alta proporción de visitantes de Madrid y Cataluña subraya la conectividad y el atractivo de la Comunidad Valenciana para estas regiones. Por otro lado, la menor afluencia de ciertas comunidades podría indicar oportunidades para estrategias de promoción turística más específicas en esas áreas.

### Estancia media de los viajeros por provincias, residencia y meses

**Columnas**

- Povincia: Provincias de destino
- Residencia: Residencia (extranjeros o no)
- Meses: Mes
- Total: Estancia media en días

In [ ]:
dataframes['df2']

Filtramos los datos para la Comunidad Valenciana seleccionando las provincias que la forman: Alicante, Valencia y Castellón

In [ ]:
# Lo que nos interesa
df_new2 = pd.concat([limpiar_datos(dataframes['df2'], destiny = 'Alicante'),
                    limpiar_datos(dataframes['df2'], destiny = 'Valencia'),
                    limpiar_datos(dataframes['df2'], destiny = 'Castellón')])

print("Total: dias de estancia medios por mes\n")
df_new2

Una vez tenemos lo que nos interesa se analizan estos datos en función de si los viajeros residen en España, son extranjeros y el total.

In [ ]:
# Residentes
R = df_new2[(df_new2["Destino"]=="Alicante/Alacant")&(df_new2['Origen']=='Residentes en España')][['Meses', 'Total']]
R_t = R['Total'].str.replace(',', '.').astype(float).to_list()[1:]
Mes = R['Meses'].to_list()[1:]
# No residentes
NR = df_new2[(df_new2["Destino"]=="Alicante/Alacant")&(df_new2['Origen']=='No residentes en España')][['Meses', 'Total']]
NR_t = NR['Total'].str.replace(',', '.').astype(float).to_list()[1:]
# Total
T = df_new2[(df_new2["Destino"]=="Alicante/Alacant")&(df_new2['Origen']=='Total')][['Meses', 'Total']]
T_t = T['Total'].str.replace(',', '.').astype(float).to_list()[1:]

In [ ]:
# Crear el gráfico de líneas
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Mes,
    y=R_t,
    mode='lines+markers',
    name='Residentes en España'
))

fig.add_trace(go.Scatter(
    x=Mes,
    y=NR_t,
    mode='lines+markers',
    name='No residentes en España'
))

fig.add_trace(go.Scatter(
    x=Mes,
    y=T_t,
    mode='lines+markers',
    name='Total'
))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Estancia Media de los Viajeros por Origen en Alicante',
    xaxis_title='Mes',
    yaxis_title='Estancia Media',
    width=1000,
    height=600
)

# Mostrar el gráfico
fig.show()

La duración media de los viajes a la Comunidad Valenciana en 2022 presenta diferencias notables entre los residentes en España y los no residentes.

Para los residentes en España, la duración media de sus viajes varía a lo largo del año. Los meses con la duración más corta son enero y febrero, con una media de 2.27 y 2.15 días respectivamente. A medida que avanza el año, la duración aumenta, alcanzando su pico en agosto con 3.79 días. Luego disminuye ligeramente en los meses siguientes, cerrando el año con una media de 3.00 días en diciembre.

Por otro lado, los no residentes presentan una duración media de viajes considerablemente más alta durante todo el año. En enero, la duración media es de 3.98 días y se mantiene relativamente constante con ligeras variaciones. El punto más alto se observa en agosto, con una duración media de 4.76 días, reflejando un aumento en el tiempo de estancia durante el verano. Diciembre cierra con una duración media de 4.40 días, más alta que la de cualquier mes de los residentes.

Al considerar el total de los viajes, que incluye tanto a residentes como a no residentes, se observa una media que se sitúa entre los valores de ambos grupos. En enero, la duración media total es de 2.95 días y sigue un patrón similar al de los residentes, con un aumento gradual a lo largo del año, alcanzando un máximo de 4.17 días en agosto y disminuyendo a 3.52 días en diciembre.

Este análisis muestra que los no residentes tienden a pasar más tiempo en la Comunidad Valenciana en comparación con los residentes, especialmente durante los meses de verano. Estos datos pueden ser útiles para planificar estrategias de turismo y servicios enfocados en maximizar la estancia de los visitantes a lo largo del año.

### Personas viajeras por edad, según destino y duración del viaje
**Columnas**
- Edad: Rango de edad
- Duración: Duración de la estancia
- Destino: Fuera de España, dentro de españa o ambas (inrtersección vacía)
- Tipo de dato: % sobre personas de la categoria (sobre el TOTAL de España, viajen o no), % sobre el total de viajeros (viajeros)
- Total: Valor del tipo de dato

In [ ]:
dataframes['df3']

En este caso se coge el año de 2019 porque para los últimos años falta información.

In [ ]:
df_new3 = limpiar_datos(dataframes['df3'], year = 2019)

df_new3

Porcentaje para cada rango de edad sobre **todos los viajeros**.

In [ ]:
tot_ed_tot = df_new3[(df_new3['Duración']=='Total')&(df_new3['Destino']=='Total')&
                     (df_new3['Tipo de dato']=='Porcentaje sobre el total de viajeros ')][['Edad',"Total"]]

tot_ed_tot

In [ ]:
por_ed_tot = tot_ed_tot['Total'].str.replace(',', '.').astype(float).to_list()
por_ed_tot = [val / por_ed_tot[0] * 100 for val in por_ed_tot[1:]]
etiquetas = ['15 a 24', '25 a 34', '35 a 44', '45 a 54', '55 a 64', '65 y más']

plotear_datos(por_ed_tot, x=etiquetas, y=por_ed_tot, title='Viajeros por Edad', xlabel=None, ylabel=None, chart_type='pie')

Para el total de viajeros, el grupo de edad con mayor proporción es el de 35 a 44 años, representando un 20.39% del total. Le siguen los grupos de 45 a 54 años (19.3%) y de 15 a 24 años (15.51%). Los grupos de 55 a 64 años y los de 65 y más años tienen proporciones similares, con un 14.72% y 14.9% respectivamente.

Porcentaje para cada rango de edad sobre los viajeros con **destino España**.

In [ ]:
tot_ed_esp = df_new3[(df_new3['Duración']=='Total')&(df_new3['Destino']=='Solo España')&
                     (df_new3['Tipo de dato']=='Porcentaje sobre el total de viajeros ')][['Edad',"Total"]]

tot_ed_esp

In [ ]:
por_ed_esp = tot_ed_esp['Total'].str.replace(',', '.').astype(float).to_list()
por_ed_esp = [val / por_ed_esp[0] * 100 for val in por_ed_esp[1:]]

plotear_datos(por_ed_esp, x=etiquetas, y=por_ed_esp, title='Viajeros a España por Edad', xlabel=None, ylabel=None, chart_type='pie')

Cuando se analiza solo a los viajeros que se desplazan dentro de España, la proporción total es del 62.61%. El grupo de edad predominante sigue siendo el de 35 a 44 años con un 12.27%, seguido de cerca por los de 45 a 54 años con un 11.86%. Los viajeros de 65 y más años representan un 11.83%, mientras que los de 15 a 24 años y de 25 a 34 años tienen porcentajes menores, con un 8.99% y 8.12% respectivamente. Los de 55 a 64 años representan un 9.54%.

Porcentaje para cada rango de edad sobre los viajeros con **destino extranjero**.

In [ ]:
tot_ed_ext = df_new3[(df_new3['Duración']=='Total')&(df_new3['Destino']=='Solo extranjero')&
                     (df_new3['Tipo de dato']=='Porcentaje sobre el total de viajeros ')][['Edad',"Total"]]

tot_ed_ext

In [ ]:
por_ed_ext= tot_ed_ext['Total'].str.replace(',', '.').astype(float).to_list()
por_ed_ext = [val / por_ed_ext[0] * 100 for val in por_ed_ext[1:]]

plotear_datos(por_ed_ext, x=etiquetas, y=por_ed_ext, title='Viajeros al Extranjero por Edad', xlabel=None, ylabel=None, chart_type='pie')

Para los viajeros que se desplazan al extranjero, el porcentaje total es de 8.23%. Dentro de este grupo, el rango de 45 a 54 años es el mayor con un 1.8%, seguido por el de 35 a 44 años con un 1.77%. Los jóvenes de 15 a 24 años tienen un 1.26%, y los de 25 a 34 años representan un 1.42%. Los viajeros de 55 a 64 años y los de 65 y más años tienen los porcentajes más bajos, con un 1.29% y 0.69% respectivamente.

Porcentaje para cada rango de edad sobre los viajeros con **destino España y extranjero**.

In [ ]:
tot_ed_amb = df_new3[(df_new3['Duración']=='Total')&(df_new3['Destino']=='España y extranjero')&
                     (df_new3['Tipo de dato']=='Porcentaje sobre el total de viajeros ')][['Edad',"Total"]]

tot_ed_amb

In [ ]:
por_ed_amb= tot_ed_amb['Total'].str.replace(',', '.').astype(float).to_list()
por_ed_amb = [val / por_ed_amb[0] * 100 for val in por_ed_amb[1:]]

plotear_datos(por_ed_amb, x=etiquetas, y=por_ed_amb, title='Viajeros a España y al Extranjero por Edad', xlabel=None, ylabel=None, chart_type='pie')

Finalmente, para los viajeros que tienen destinos tanto en España como en el extranjero, el porcentaje total es de 29.17%. Los mayores porcentajes en este grupo se encuentran en los rangos de 35 a 44 años (6.35%) y de 25 a 34 años (5.65%). Los de 45 a 54 años y de 15 a 24 años tienen proporciones similares, con un 5.64% y 5.26% respectivamente. Los viajeros de 55 a 64 años representan un 3.89%, mientras que los de 65 y más años tienen el porcentaje más bajo con un 2.38%.

El análisis revela que los viajeros de mediana edad (35 a 54 años) constituyen la mayoría de los desplazamientos tanto nacionales como internacionales. Los jóvenes de 15 a 24 años y los adultos mayores de 55 años muestran una menor proporción de viajes, especialmente al extranjero. Este patrón puede ser útil para las agencias de turismo y las autoridades de planificación para diseñar estrategias y servicios específicos que atiendan a las necesidades y preferencias de cada grupo de edad.

### Viajeros y pernoctaciones por comunidades autónomas y provincias

**Columnas**
- Totales Territoriales: 'Total Nacional' (No aporta información)
- Comunidades y Ciudades Autónomas: CA de destino
- Provincias: Provincia de destino
- Viajeros y pernoctaciones: totales
- Residencia Nivel 1: solo hacer caso a total nan
- Residencia Nivel 2: si son extranjertos o españoles
- Periodo: Meses y Año
- Total: Valor de viajeros o pernoctaciones

In [ ]:
dataframes['df4']

Cambiamos el formato de los dato, se crean dos columnas ['Mes'] y ['Año'] a partir de ['Periodo']. Además, filtramos por Comunidad Valenciana y año 2022 para tener los datos más recientes.

In [ ]:
df_new4 = pd.concat([limpiar_datos(dataframes['df4'], destiny='Valencia', year=2022),
                     limpiar_datos(dataframes['df4'], destiny='Alicante', year=2022),
                     limpiar_datos(dataframes['df4'], destiny='Castellón', year=2022)])
df_new4

Veamos el **número de viajeros** por mes que vienen a Alicante dependiendo de su residencia.

In [ ]:
# Residentes
Resi = limpiar_datos(df_new4, destiny='Alicante', origin='España')
R_viaj = Resi[Resi['Viajeros y pernoctaciones']=='Viajero']
R_viaj_t = R_viaj['Total'].str.replace('.', '').astype(float).to_list()
Mes = R_viaj['Mes'].to_list()[::-1]

# No residentes
NResi = limpiar_datos(df_new4, destiny='Alicante', origin='Extranjero')
NR_viaj = NResi[NResi['Viajeros y pernoctaciones']=='Viajero']
NR_viaj_t = NR_viaj['Total'].str.replace('.', '').astype(float).to_list()

# Total
Tot = limpiar_datos(df_new4, destiny='Alicante', origin='Total')
T_viaj = Tot[Tot['Viajeros y pernoctaciones']=='Viajero']
T_viaj_t = T_viaj['Total'].str.replace('.', '').astype(float).to_list()

In [ ]:
# Crear el gráfico de líneas
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Mes,
    y=R_viaj_t[::-1],
    mode='lines+markers',
    name='Residentes en España'
))

fig.add_trace(go.Scatter(
    x=Mes,
    y=NR_viaj_t[::-1],
    mode='lines+markers',
    name='No residentes en España'
))

fig.add_trace(go.Scatter(
    x=Mes,
    y=T_viaj_t[::-1],
    mode='lines+markers',
    name='Total'
))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Numero de Viajeros a Alicante',
    xaxis_title='Mes',
    yaxis_title='Número de Viajeros',
    width=1000,
    height=600
)

# Mostrar el gráfico
fig.show()

El flujo de viajeros residentes en la Comunidad Valenciana tiene un incremento notable durante los meses de verano. Enero presenta el número más bajo de viajeros con 75,778, mientras que agosto muestra el pico más alto con 341,418 viajeros. Este aumento significativo en los meses de verano refleja la tendencia de los residentes a viajar más durante las vacaciones estivales. Otros meses con alto número de viajeros son julio (319,351) y septiembre (209,071), mientras que los meses de invierno y principios de año presentan cifras menores.

Para los viajeros no residentes, el patrón es similar en términos de aumento durante los meses de verano, aunque con diferentes magnitudes. Agosto es el mes con mayor afluencia de no residentes, alcanzando los 219,568 viajeros. Enero tiene el menor número de viajeros no residentes con 49,765. Al igual que los residentes, los no residentes muestran un incremento durante los meses de primavera y verano, con julio (217,146) y septiembre (210,661) siendo meses destacados.

Al considerar el total de viajeros, que incluye tanto a residentes como a no residentes, los meses de verano son nuevamente los más concurridos. Agosto se destaca con una afluencia total de 560,986 viajeros, seguido de julio con 536,497 y septiembre con 419,732. Enero presenta el menor número total de viajeros con 125,543. Estos datos indican que la Comunidad Valenciana es un destino altamente preferido durante el verano, atrayendo tanto a residentes como a no residentes.

Este análisis revela una clara estacionalidad en los viajes a la Comunidad Valenciana, con una preferencia marcada por los meses de verano. Las políticas de promoción turística podrían aprovechar esta tendencia para maximizar la afluencia en otras temporadas del año, ofreciendo incentivos y actividades que atraigan a los viajeros durante los meses menos concurridos.

Vemaos ahora el número de **pernoctaciones**.

In [ ]:
# Residentes
Resi = limpiar_datos(df_new4, destiny='Alicante', origin='Residentes en España')
R_per = Resi[Resi['Viajeros y pernoctaciones']=='Pernoctaciones']
R_per_t = R_per['Total'].str.replace('.', '').astype(float).to_list()
Mes = R_per['Mes'].to_list()[::-1]
# No residentes
NResi = limpiar_datos(df_new4, destiny='Alicante', origin='Residentes en el Extranjero')
NR_per = NResi[NResi['Viajeros y pernoctaciones']=='Pernoctaciones']
NR_per_t = NR_per['Total'].str.replace('.', '').astype(float).to_list()
# Total
Tot = limpiar_datos(df_new4, destiny='Alicante', origin='Total')
T_per = Tot[Tot['Viajeros y pernoctaciones']=='Pernoctaciones']
T_per_t = T_per['Total'].str.replace('.', '').astype(float).to_list()

In [ ]:
# Crear el gráfico de líneas
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Mes,
    y=R_per_t[::-1],
    mode='lines+markers',
    name='Residentes en España'
))

fig.add_trace(go.Scatter(
    x=Mes,
    y=NR_per_t[::-1],
    mode='lines+markers',
    name='No residentes en España'
))

fig.add_trace(go.Scatter(
    x=Mes,
    y=T_per_t[::-1],
    mode='lines+markers',
    name='Total'
))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Número de Pernoctaciones en Alicante',
    xaxis_title='Mes',
    yaxis_title='Número de Pernoctaciones',
    width=1000,
    height=600
)

# Mostrar el gráfico
fig.show()

Las pernoctaciones de los residentes en la Comunidad Valenciana reflejan un patrón de aumento durante los meses de verano. Enero tiene el menor número de pernoctaciones con 171,987, mientras que agosto alcanza el pico con 1,292,954 pernoctaciones. Otros meses de alta actividad incluyen julio (1,119,077) y septiembre (726,002), mostrando una clara preferencia por las vacaciones de verano. Los meses de invierno y principios de año tienen cifras significativamente menores.

Para los no residentes, el patrón es similar en términos de estacionalidad, aunque las cifras absolutas difieren. Agosto nuevamente es el mes con mayor número de pernoctaciones, con 1,045,358, seguido de cerca por septiembre (954,588) y julio (953,197). Enero presenta el menor número de pernoctaciones con 198,286. Los meses de primavera y verano también muestran un aumento significativo en las pernoctaciones.

Al considerar el total de pernoctaciones, que incluye tanto a residentes como a no residentes, los meses de verano se destacan notablemente. Agosto tiene la mayor cantidad de pernoctaciones totales con 2,338,312, seguido por julio con 2,072,274 y septiembre con 1,680,590. Enero y febrero presentan las cifras más bajas, con 370,273 y 595,410 respectivamente.

Este análisis muestra una clara estacionalidad en las pernoctaciones, con una concentración significativa durante los meses de verano. Tanto residentes como no residentes prefieren la Comunidad Valenciana como destino turístico principalmente en agosto, julio y septiembre. Las políticas de promoción turística podrían enfocarse en equilibrar esta distribución a lo largo del año, incentivando visitas durante las temporadas bajas con ofertas especiales y eventos atractivos para mantener un flujo constante de turistas durante todo el año.

### Viajes, pernoctaciones y duración media por nivel de formación alcanzado de los viajeros

**Columnas**
- Nivel de formación: Nivel de formación
- Concepto turístico: Viajes, pernoctaciones y duración media de los viajes
- TIPO DE DATO:
  - Valor absoluto - total
  - Distribución porcentual - según el tipo de formación, en cada periodo
  - Acumulado en lo que va de año (si T2, T1+T2)
  - Distribución porcentual del acumulado (divide como el otro, T1 da igual que sin acuimular, el resto no)
  - Variación anual, comparación con el año anterior:(año-año anterior)/(año anterior)
  - Variación anual del acumulado en lo que va de año (como arriba)
- Periodo en trimestres
- Total: Valor del concepto turístico

In [ ]:
dataframes['df5']

Creamos la columna ['Año'] a partir del periodo y la columna ['Trimestre']. De nuevo nos quedamos con el año 2022 y todas las columnas excepto la de ['Periodo].

In [ ]:
df_new5 = limpiar_datos(dataframes['df5'], year=2022)
df_new5

Veamos el número de **viajes** que hacen según el nivel de formación en cada temporada.

In [ ]:
df_form_viaj = df_new5[(df_new5['Concepto turístico']=='Viajes')&(df_new5["Tipo de dato"]=="Valor absoluto")][['Nivel de formación', 'Trimestre', 'Total']]
df_form_viaj['Total'] = df_form_viaj['Total'].str.replace('.', '').astype(float)


In [ ]:
pivot_df = df_form_viaj.pivot(index='Nivel de formación', columns='Trimestre', values='Total')
pivot_df

In [ ]:
# Crear el heatmap
fig = go.Figure(data=go.Heatmap(
    z=pivot_df.values,
    x=pivot_df.columns,
    y=pivot_df.index,
    colorscale='Viridis',
    text=pivot_df.values,  # Añadir los valores para anotaciones
    texttemplate="%{text}",  # Formato de las anotaciones
    hoverinfo="z"  # Información a mostrar al pasar el ratón
))

# Añadir título y etiquetas a los ejes
fig.update_layout(
    title='Número de Viajes por Formación y Temporada',
    xaxis_title='Temporada',
    yaxis_title='Formación'
)

# Mostrar el gráfico
fig.show()

Los viajes realizados por personas con educación primaria e inferior muestran un aumento en la temporada T3 (julio, agosto y septiembre) con 3,405,252 viajes, siendo este el trimestre con mayor actividad. La temporada T1 (enero, febrero y marzo) tiene el menor número de viajes con 1,624,337.

Para aquellos con educación secundaria de primera etapa, la temporada T3 también es la más activa con 9,302,333 viajes. La temporada T1 tiene el menor número con 5,903,426 viajes, pero aún así, representa un volumen considerable de desplazamientos.

Las personas con educación secundaria de segunda etapa realizan la mayoría de sus viajes en la temporada T3, alcanzando los 11,631,909. El trimestre menos activo es T1 con 5,941,816 viajes.

Los individuos con educación superior son los más viajeros en general, con un notable incremento en la temporada T3 que registra 23,867,824 viajes. La temporada T1 muestra el menor número de viajes con 14,350,162.

En conjunto, la tendencia general sigue mostrando que la temporada T3 es la más activa con un total de 48,207,318 viajes, mientras que T1 es la menos activa con 27,819,742 viajes.

Este análisis muestra una clara preferencia por viajar durante la temporada T3, que coincide con los meses de verano en el hemisferio norte, cuando muchas personas toman vacaciones. Las diferencias en los patrones de viaje según el nivel de formación pueden deberse a factores como disponibilidad de tiempo libre, ingresos y preferencias personales. Estos datos pueden ser útiles para planificar servicios y campañas de turismo que se adapten a las necesidades de cada grupo educativo durante diferentes épocas del año.

Veamos ahora la **duración media** del viaje según el nivel de formación para cada temporada.

In [ ]:
df_form_dura = df_new5[(df_new5['Concepto turístico']=='Duración media de los viajes')&(df_new5["Tipo de dato"]=="Valor absoluto")][['Nivel de formación', 'Trimestre', 'Total']]
df_form_dura['Total'] = df_form_dura['Total'].str.replace(',', '.').astype(float)

In [ ]:
pivot_df = df_form_dura.pivot(index='Nivel de formación', columns='Trimestre', values='Total')
pivot_df

In [ ]:
# Crear el heatmap
fig = go.Figure(data=go.Heatmap(
    z=pivot_df.values,
    x=pivot_df.columns,
    y=pivot_df.index,
    colorscale='Viridis',
    text=pivot_df.values,  # Añadir los valores para anotaciones
    texttemplate="%{text}",  # Formato de las anotaciones
    hoverinfo="z"  # Información a mostrar al pasar el ratón
))

# Añadir título y etiquetas a los ejes
fig.update_layout(
    title='Número de Viajes por Formación y Temporada',
    xaxis_title='Temporada',
    yaxis_title='Formación'
)

# Mostrar el gráfico
fig.show()

Las personas con educación primaria e inferior tienen la duración media de viaje más larga en la temporada T3 (julio, agosto y septiembre), con 8.91 días. Las demás temporadas tienen duraciones medias menores, siendo T1 y T4 las más cortas con aproximadamente 4.27 y 4.23 días respectivamente.

Para aquellos con educación secundaria de primera etapa, la duración media más larga también se encuentra en T3, con 6.47 días. Las duraciones en T1 y T4 son más cortas, alrededor de 3.41 y 3.65 días respectivamente, con una ligera variación en T2 a 3.63 días.

Las personas con educación secundaria de segunda etapa presentan una duración media de 6.29 días en T3. Las otras temporadas tienen duraciones similares, alrededor de 3.10 a 3.47 días.

Los individuos con educación superior viajan por una media de 5.99 días en T3, mientras que las duraciones en T1, T2 y T4 son bastante consistentes, alrededor de 3.10 a 3.29 días.

En general, la duración media de los viajes es más larga en la temporada T3, con 6.36 días. Las otras temporadas presentan duraciones más cortas y bastante similares, con T1 a 3.24 días, T2 a 3.47 días y T4 a 3.45 días.

Este análisis revela que, independientemente del nivel de formación, la duración media de los viajes es más prolongada en la temporada T3, que coincide con las vacaciones de verano en el hemisferio norte. Las políticas de turismo pueden aprovechar este dato para diseñar paquetes y ofertas que incentiven viajes más largos durante otras temporadas, equilibrando así la distribución de turistas a lo largo del año.

### Viajes, pernoctaciones, duración media y gasto por comunidad autónoma de residencia de los viajeros

**Columnas**
- Comunidad autónoma de residencia: CA de residencia
- Concepto turístico: Viajes, Duración media de los viajes, Pernoctaciones, Gasto total, Gasto medio por persona, Gasto medio diario por persona
- TIPO DE DATO:
  - Valor absoluto - total
  - Distribución porcentual
  - Acumulado en lo que va de año
  - Distribución porcentual del acumulado
  - Variación anual
  - Variación anual del acumulado en lo que va de año
- Periodo: Periodo en trimestres y Año
- Total: Valor del concepto turístico

In [ ]:
dataframes['df6']

Nos quedamos con el año 2022 y los valores absolutos.

In [ ]:
df_new6 = limpiar_datos(dataframes['df6'], year=2022)
df_new6

Analizamos el gasto medio diario por persona según su CA de residencia para cada trimestre.

In [ ]:
# T1
T1_gasto = df_new6[(df_new6['Concepto turístico']=='Gasto medio diario por persona')&
                   (df_new6['Trimestre']=='Trimestre 1')&(df_new6["Tipo de dato"]=="Valor absoluto")][['Origen','Total']]
T1_gasto_t = T1_gasto['Total'].str.replace('.', '').str.replace(',', '.').astype(float).to_list()
T1_gasto_t[-2] = round((T1_gasto_t[-1]+T1_gasto_t[-2])/2,2)
T1_gasto_t.pop()

# T2
T2_gasto = df_new6[(df_new6['Concepto turístico']=='Gasto medio diario por persona')&
                   (df_new6['Trimestre']=='Trimestre 2')&(df_new6["Tipo de dato"]=="Valor absoluto")][['Origen','Total']]
T2_gasto_t = T2_gasto['Total'].str.replace('.', '').str.replace(',', '.').astype(float).to_list()
T2_gasto_t[-2] = round((T2_gasto_t[-1]+T2_gasto_t[-2])/2,2)
T2_gasto_t.pop()

# T3
T3_gasto = df_new6[(df_new6['Concepto turístico']=='Gasto medio diario por persona')&
                   (df_new6['Trimestre']=='Trimestre 3')&(df_new6["Tipo de dato"]=="Valor absoluto")][['Origen','Total']]
T3_gasto_t = T3_gasto['Total'].str.replace('.', '').str.replace(',', '.').astype(float).to_list()
T3_gasto_t[-2] = round((T3_gasto_t[-1]+T3_gasto_t[-2])/2,2)
T3_gasto_t.pop()

# T4
T4_gasto = df_new6[(df_new6['Concepto turístico']=='Gasto medio diario por persona')&
                   (df_new6['Trimestre']=='Trimestre 4')&(df_new6["Tipo de dato"]=="Valor absoluto")][['Origen','Total']]
T4_gasto_t = T4_gasto['Total'].str.replace('.', '').str.replace(',', '.').astype(float).to_list()
T4_gasto_t[-2] = round((T4_gasto_t[-1]+T4_gasto_t[-2])/2,2)
T4_gasto_t.pop()

In [ ]:
path = 'spain'
title = "Mapa gasto medio diario por persona en T1"
spain_heat_map(T1_gasto_t[1:], path, "bluyl", title, plot_type="plotly")

title = "Mapa gasto medio diario por persona en T2"
spain_heat_map(T2_gasto_t[1:], path, "bluyl", title, plot_type="plotly")

title = "Mapa gasto medio diario por persona en T3"
spain_heat_map(T3_gasto_t[1:], path, "bluyl", title, plot_type="plotly")

title = "Mapa gasto medio diario por persona en T4"
spain_heat_map(T4_gasto_t[1:], path, "bluyl", title, plot_type="plotly")

En la temporada T1, el gasto medio más alto proviene de Canarias, con 87.94 euros diarios, seguido por el País Vasco con 80.16 euros y Murcia con 78.85 euros. La Rioja tiene el gasto medio más bajo con 56.72 euros, y la Comunitat Valenciana también se encuentra en el rango bajo con 60.83 euros.

Durante la temporada T2, las Islas Baleares tienen el gasto medio más alto con 83.12 euros, seguido por Cataluña con 86.49 euros y la Comunidad de Madrid con 81.65 euros. El gasto medio más bajo se observa en Ceuta y Melilla con 66.81 euros.

La temporada T3 muestra una disminución general en el gasto medio. Las Islas Baleares tienen el gasto medio más bajo con 47.97 euros, seguido por La Rioja con 47.95 euros y el País Vasco con 49.17 euros. Por otro lado, Canarias mantiene un gasto relativamente alto con 66.80 euros.

En la temporada T4, el gasto medio más alto se registra nuevamente en Canarias con 103.68 euros, seguido por las Islas Baleares con 96.11 euros y Cataluña con 90.99 euros. El gasto medio más bajo se observa en Castilla-La Mancha con 65.70 euros.

El análisis muestra que el gasto medio diario varía significativamente no solo entre las comunidades, sino también entre las distintas temporadas. Canarias y las Islas Baleares tienden a tener los gastos medios más altos en la mayoría de las temporadas, especialmente en T4. En contraste, La Rioja y la Comunitat Valenciana suelen tener gastos más bajos. Estos datos pueden ser útiles para la planificación de estrategias turísticas regionales, adaptando las ofertas y promociones según las temporadas y los hábitos de gasto de los viajeros de diferentes comunidades.

Analicemos ahora la duración media de los viajes según la comunidad de residendencia para cada trimestre.

In [ ]:
# T1
T1_dura = df_new6[(df_new6['Concepto turístico']=='Duración media de los viajes')&
                  (df_new6['Trimestre']=='Trimestre 1')&(df_new6["Tipo de dato"]=="Valor absoluto")][['Origen','Total']]
T1_dura_t = T1_dura['Total'].str.replace('.', '').str.replace(',', '.').astype(float).to_list()
T1_dura_t[-2] = (T1_dura_t[-1]+T1_dura_t[-2])/2
T1_dura_t.pop()

# T2
T2_dura = df_new6[(df_new6['Concepto turístico']=='Duración media de los viajes')&
                  (df_new6['Trimestre']=='Trimestre 2')&(df_new6["Tipo de dato"]=="Valor absoluto")][['Origen','Total']]
T2_dura_t = T2_dura['Total'].str.replace('.', '').str.replace(',', '.').astype(float).to_list()
T2_dura_t[-2] = (T2_dura_t[-1]+T2_dura_t[-2])/2
T2_dura_t.pop()

# T3
T3_dura = df_new6[(df_new6['Concepto turístico']=='Duración media de los viajes')&
                  (df_new6['Trimestre']=='Trimestre 3')&(df_new6["Tipo de dato"]=="Valor absoluto")][['Origen','Total']]
T3_dura_t = T3_dura['Total'].str.replace('.', '').str.replace(',', '.').astype(float).to_list()
T3_dura_t[-2] = (T3_dura_t[-1]+T3_dura_t[-2])/2
T3_dura_t.pop()

# T4
T4_dura = df_new6[(df_new6['Concepto turístico']=='Duración media de los viajes')&
                  (df_new6['Trimestre']=='Trimestre 4')&(df_new6["Tipo de dato"]=="Valor absoluto")][['Origen','Total']]
T4_dura_t = T4_dura['Total'].str.replace('.', '').str.replace(',', '.').astype(float).to_list()
T4_dura_t[-2] = (T4_dura_t[-1]+T4_dura_t[-2])/2
T4_dura_t.pop()


In [ ]:
path = 'spain'
title = "Mapa duración media del viaje en T1"
spain_heat_map(T1_dura_t[1:], path, "bluyl",title, plot_type="plotly")

title = "Mapa duración media del viaje en T2"
spain_heat_map(T2_dura_t[1:], path, "bluyl",title, plot_type="plotly")

title = "Mapa duración media del viaje en T3"
spain_heat_map(T3_dura_t[1:], path, "bluyl",title, plot_type="plotly")

title = "Mapa duración media del viaje en T4"
spain_heat_map(T4_dura_t[1:], path, "bluyl",title, plot_type="plotly")

En la temporada T1, Ceuta y Melilla tienen la duración media más larga con 5.835 días, seguida por las Islas Baleares con 4.640 días y Canarias con 3.980 días. Murcia presenta la duración media más corta con 2.590 días.

Durante la temporada T2, Ceuta y Melilla nuevamente registran la duración más larga con 6.25 días, seguidas por el País Vasco con 4.62 días y Canarias con 4.44 días. La duración más corta se observa en Aragón y Asturias, ambos con 2.92 días.

En la temporada T3, Ceuta y Melilla tienen la mayor duración media de los viajes con 8.615 días, seguidas por el País Vasco con 8.540 días y la Comunidad de Madrid con 7.860 días. Las Islas Baleares tienen una duración media de 5.420 días, que es relativamente baja en comparación con otras regiones en esta temporada.

En la temporada T4, la duración media más larga se registra en Ceuta y Melilla con 3.845 días, seguidas por las Islas Baleares con 4.250 días y Canarias con 3.920 días. La Rioja tiene la duración media más corta con 2.810 días.

Este análisis muestra que la duración media de los viajes es más larga en la temporada T3 para todas las comunidades, coincidiendo con las vacaciones de verano. Ceuta y Melilla destacan por tener las duraciones más largas en varias temporadas, mientras que regiones como Murcia y La Rioja suelen tener duraciones más cortas. Estos patrones reflejan las preferencias de viaje estacionales y regionales, y pueden ser útiles para la planificación de estrategias turísticas que consideren la duración media de las estancias según la comunidad de residencia.

### Viajes, pernoctaciones, duración media y gasto por destino principal (extranjero y comunidades autónomas)

**Columnas**
- Destino: CA de destino
- Concepto turístico: Viajes, Duración media de los viajes, Pernoctaciones, Gasto total, Gasto medio por persona y Gasto medio diario por persona
- Tipo de dato: Valor absoluto, Distribución porcentual y Variación anual
- Periodo: Año
- Total: Valor del concepto turístico



In [ ]:
dataframes['df7']

Nos quedamos con el año 2022 y los datos refentes a la Comunidad Valenciana.

In [ ]:
df_new7 = limpiar_datos(dataframes['df7'], year=2022, destiny='Valenciana')
df_new7

In [ ]:
d_cv = df_new7[df_new7['Tipo de dato']=='Valor absoluto'][['Concepto turístico','Total']]
d_cv

En total, se registraron 17,932,009 viajes a la Comunidad Valenciana, lo que indica un flujo significativo de turistas hacia la región. La duración media de los viajes fue de 4.69 días, lo que sugiere que los visitantes permanecen en la región por un período moderado, disfrutando de sus diversas atracciones y servicios. Se contabilizaron 84,030,696 pernoctaciones, lo que demuestra una alta ocupación de alojamientos turísticos durante el año. Esto es un indicador clave de la actividad turística y su impacto en la industria de la hospitalidad. El gasto total de los turistas ascendió a 4,193,264.76 euros. Esta cifra refleja la contribución económica significativa del turismo a la economía regional. El gasto medio por persona fue de 233.84 euros, lo que proporciona una visión del gasto individual de los turistas durante su estancia. El gasto medio diario por persona fue de 49.9 euros. Este dato es crucial para entender el poder adquisitivo de los turistas y su impacto diario en la economía local.

Estos datos subrayan la importancia del turismo para la Comunidad Valenciana, destacando no solo el volumen de turistas y pernoctaciones, sino también el impacto económico considerable que genera esta actividad. La combinación de una duración media de viajes moderada y un gasto diario significativo sugiere que la región es capaz de atraer y mantener a turistas que contribuyen de manera sustancial a su economía. Esto también puede orientar futuras estrategias de marketing y desarrollo turístico para maximizar el potencial de la región.

### Viajes, pernoctaciones, duración media y gasto por tipo de destino principal

**Columnas**
- Tipo de destino: Total, Extranjero, España, Dentro de la Comunidad Autónoma,
 Dentro de la provincia, A otra Comunidad Autónoma
- Concepto turístico: Viajes, Duración media de los viajes, Pernoctaciones, Gasto total,
 Gasto medio por persona, Gasto medio diario por persona
- Tipo de dato: Valor absoluto
- Periodo: Año
- Total: Valor del concepto turístico


In [ ]:
dataframes['df8']

Elegimos el año 2022.

In [ ]:
df_new8 = limpiar_datos(dataframes['df8'], year=2022)
df_new8 = df_new8[df_new8['Tipo de dato']=='Valor absoluto']
df_new8

In [ ]:
# Pivotar el DataFrame
df_pivot = df_new8.pivot(index='Destino', columns='Concepto turístico', values='Total')
df_pivot

Los viajes al extranjero tienen la mayor duración media (8.53 días) y el gasto medio por persona más alto (920.98 euros). En contraste, los viajes dentro de la provincia tienen la menor duración media (2.93 días) y el gasto medio por persona más bajo (97.59 euros).

El total de gasto más alto se registra en viajes dentro de España (35,431,861.07 euros), mientras que los viajes a otra comunidad autónoma y dentro de la comunidad autónoma también tienen altos valores de pernoctaciones y viajes.

El gasto medio diario es más alto para los viajes al extranjero (108.01 euros), seguido por los viajes a otra comunidad autónoma (68.69 euros).

Los viajes dentro de la provincia son más baratos y cortos, lo que los hace atractivos para escapadas rápidas y económicas. Por el contrario, los viajes al extranjero son más prolongados y costosos. Los viajes dentro de España representan una parte significativa del gasto turístico total, mostrando una fuerte preferencia por el turismo doméstico y la exploración de diferentes comunidades autónomas.

Estos datos reflejan las diferentes tendencias y comportamientos de los turistas según el destino, proporcionando información valiosa para la planificación y promoción turística en diversas regiones.



### Turismo receptor: número de turistas y pernoctaciones por medios de transporte, periodo y turistas/pernoctaciones

**Columnas**
- Medios de transporte: Aeropuerto, Tren, Puerto, Carretera, Total
- Turistas/pernoctaciones: Número de llegadas, Número de pernoctaciones
- Periodo: Año
- Total: Valor del concepto turístico

In [ ]:
url = "https://www.ine.es/jaxi/files/_px/csv_bdsc/t35/p011/rev19/serie/02002.csv"
df9 = pd.read_csv(url, sep = ";")
df9

Filtramos por el año 2022.

In [ ]:
# Lo que nos interesa
df_new9 = df9[(df9["periodo"]==2022)][["Medios de transporte", "Turistas/pernoctaciones", "Total"]]

print("Total: número de viajeros o número total de pernoctaciones")
df_new9

La mayoría de los turistas llegan a España por vía aérea, representando una abrumadora mayoría con 59,307,849 llegadas. Esto destaca la importancia de los aeropuertos como principal punto de entrada para los turistas. Las llegadas por carretera también son significativas, sumando 10,964,073, lo que indica que una parte considerable de turistas, probablemente provenientes de países vecinos, eligen viajar en coche. En comparación, las llegadas por puerto y tren son mucho menores, con 1,152,225 y 235,135 respectivamente, reflejando un uso más limitado de estos medios para el turismo.

Las pernoctaciones siguen una tendencia similar a las llegadas. Los turistas que llegan por vía aérea generan la mayor cantidad de pernoctaciones, con 444,576,423 noches. Las pernoctaciones asociadas con llegadas por carretera alcanzan 84,462,626, lo que nuevamente resalta la importancia de los viajes terrestres. Las pernoctaciones por puerto y tren son mucho menores, con 7,255,340 y 1,317,386 respectivamente.

Los aeropuertos no solo son el principal punto de entrada para los turistas, sino que también están asociados con la mayor cantidad de pernoctaciones. Esto sugiere que los turistas que llegan por aire tienden a quedarse más tiempo, lo que puede estar relacionado con el tipo de turistas que prefieren viajes internacionales y probablemente tienen una mayor capacidad de gasto.

Aunque los viajes por carretera son menos numerosos que los aéreos, siguen siendo una parte importante del turismo en España. Los turistas que llegan en coche también contribuyen significativamente a las pernoctaciones, aunque en menor medida.

El uso de puertos y trenes para llegar a España es bastante limitado en comparación con los aviones y coches. Esto puede deberse a varios factores, como la comodidad, la velocidad y la infraestructura disponible.

En total, España recibió 71,659,282 turistas en 2022, con una contribución sustancial de 537,611,775 pernoctaciones. Estos datos subrayan la importancia del turismo para la economía española, con una clara preferencia por los viajes aéreos y terrestres.

## Conclusiones

En 2022, los datos turísticos de la Comunidad Valenciana revelan diversos perfiles de viajeros y patrones de comportamiento. Los viajes realizados a otras comunidades autónomas tienen una duración media de 4.76 días y un gasto medio diario de 68.69 euros por persona, lo que se traduce en un gasto total de 327.07 euros por persona. Este perfil de viajeros tiende a realizar viajes moderadamente largos y a gastar más en promedio, prefiriendo destinos fuera de su comunidad pero dentro de España.

Por otro lado, los viajes dentro de la comunidad autónoma son más cortos, con una duración media de 3.1 días y un gasto medio diario de 42.83 euros, resultando en un gasto total de 132.95 euros por persona. Estos viajeros suelen realizar escapadas breves y económicas dentro de su propia comunidad. Los viajes dentro de la provincia tienen una duración aún menor, de 2.93 días, y un gasto medio diario de 33.33 euros, reflejando una preferencia por destinos muy cercanos y un gasto más conservador.

Los viajeros internacionales presentan un perfil distinto, con estancias más largas, de 8.53 días en promedio, y un gasto medio diario de 108.01 euros. Esto se traduce en un gasto total por persona de 920.98 euros. Estos viajeros tienen una mayor capacidad adquisitiva y prefieren estancias prolongadas.

En general, la duración media de los viajes dentro de España es de 3.92 días, con un gasto medio diario de 58.25 euros, mientras que el gasto total por persona asciende a 228.22 euros. Los datos también muestran que la temporada de verano (T3) es la más activa, con la mayor duración de viajes y gasto, coincidiendo con las vacaciones estivales.

El análisis de los datos subraya la importancia del turismo para la Comunidad Valenciana, tanto a nivel nacional como internacional. Los viajeros internacionales, aunque menos numerosos, aportan un gasto significativo debido a sus estancias más largas y mayor gasto diario. Los viajes dentro de la comunidad y la provincia son más frecuentes pero con menor impacto económico individualmente.

Para maximizar el potencial turístico de la región, se pueden desarrollar estrategias de marketing que atraigan a los viajeros de mayor gasto durante las temporadas bajas y crear ofertas especiales para los viajeros locales que prefieren escapadas rápidas y económicas. Además, mejorar la infraestructura turística para acomodar a los viajeros internacionales y promover el turismo dentro de la comunidad autónoma puede ayudar a equilibrar la afluencia turística a lo largo del año. Estas estrategias pueden contribuir a un desarrollo turístico más sostenible y beneficioso para la Comunidad Valenciana.